In [ ]:
!pip install selenium 

In [ ]:
!pip install fake_useragent --q

In [ ]:
!pip install html5lib --q

In [22]:
import requests

print(requests.__version__)


2.31.0


In [23]:
!pip install --upgrade requests --user

In [ ]:
!pip install keyboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.0 MB/s eta 0:00:00


In [1]:
import time
from datetime import datetime
from selenium import webdriver
##import keyboard
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
from time import gmtime, strftime
import  html5lib

In [2]:
from random import randint,shuffle
from time import sleep

In [3]:
def extrair_numero(texto):
    # Divide a string pelos caracteres de barra ("/")
    partes = texto.split('/')

    # Procura o número na lista resultante
    for parte in partes:
        if parte.isdigit():
            return int(parte)

In [4]:
def ajusta_coluna(df):

    df_columns = [' '.join(col).strip() for col in df.columns]
    new_columns = []

    for col in df_columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)

    return new_columns

In [5]:
def get_countries(url_df):

  #Arrumar Países
  ua = UserAgent()
  header = {'User-Agent':str(ua.random)}

  data = requests.get(url_df, headers=header)
  soup =  BeautifulSoup(data.text)
  champ_table = soup.select('table.stats_table')[0]

  links = champ_table.find_all('a')
  links = [l.get('href') for l in links] #href é a propriedade que contem o link de acesso

  name_country = [l.split('/')[-1].replace('-Football','').replace('-',' ') for l in links if '/country/' in l]
  sigla_country = [l.split('/')[3] for l in links if '/country/' in l]

  teste = set(zip(sigla_country,name_country))

  dict_country = {}
  for tupla in teste:
    chave, valor = tupla
    dict_country[chave] = valor

  dict_country['N/I'] = 'N/I'

  return dict_country


In [6]:
def get_comps_coverage():
    url_df = 'https://fbref.com/en/about/coverage'
    df = pd.read_html(url_df)[0]
    df['Tier'] = df['Tier'].replace({1:'First Division',2:'Second Division',3:'Third Division',4:'Fourth Division',5:'Fifth Division',np.NaN:'Cup'})
    df['Country'] = df['Country'].replace({np.NaN:'N/I'})
    df['Country'] = [l.split(' ')[1] if l != 'N/I' else 'N/I' for l in df['Country']]

    dict_country = get_countries(url_df)


    df['Country'] = df['Country'].map(dict_country)

    return df

In [7]:
def get_comps_link(list_country = []):

    df_all = get_comps_coverage()



    url = 'https://fbref.com/en/about/coverage'
    ua = UserAgent()
    header = {'User-Agent':str(ua.random)}

    data = requests.get(url, headers=header)
    soup =  BeautifulSoup(data.text)
    champ_table = soup.select('table.stats_table')[0]

    links = champ_table.find_all('a')
    links = [l.get('href') for l in links] #href é a propriedade que contem o link de acesso
    links = [l for l in links if '/comps/' in l]


    df_all['league_number'] = np.NaN
    df_all['all_seasons_link'] = np.NaN
    df_all['stats_link'] = np.NaN

    for i in range(len(links)):
        num_comps = extrair_numero(links[i])
        if num_comps is not None:
            df_all.iloc[i,7] = int(num_comps)
            df_all.iloc[i,8] = links[i]

            padrao = r'history/(.*?)-Seasons'
            # Tenta encontrar o padrão no texto
            resultado = re.search(padrao, links[i])

            if resultado:
                texto_extraido = resultado.group(1)

                df_all.iloc[i,9] = '/en/comps/'+str(num_comps)+'/()/()-'+texto_extraido+'-Stats'

        else:
            df_all.iloc[i,7] = np.NaN

    df_all = df_all[df_all['Country'].isin(list_country)].reset_index().drop(columns = ['index'])

    return df_all

In [8]:
def get_table_stats(list_country = None,list_league = None):

    df_all = get_comps_link()
    stats_list = ['Squad Standard Stats','Squad Playing Time','Squad Goalkeeping','Squad Shooting','Squad Miscellaneous Stats']

    df_list_1 = []
    df_list_2 = []
    df_list_3 = []
    df_list_4 = []
    df_list_5 = []

    for i,l in df_all.iterrows():
        if l['Tier'] != 'Cup' and l['Gender'] == 'M' :

            delay = randint(5,20)

            stats_link = l['stats_link']
            season = l['Last Season']

            acess_link = r'https://fbref.com' + stats_link.replace('()',season)


            sleep(delay)

            ua = UserAgent()
            header = {'User-Agent':str(ua.random)}
            data = requests.get(acess_link, headers=header)

            for stats in stats_list:

                try:

                    delay = randint(5,20)
                    sleep(delay)
                    df = pd.read_html(requests.get(acess_link, headers=header).text, match = stats)[0]
                    df['league'] = l['Comp']
                    df['season'] = season
                    if stats == 'Squad Standard Stats':
                        df_list_1.append(df)
                        print(stats)
                    if stats == 'Squad Playing Time':
                        df_list_2.append(df)
                        print(stats)
                    if stats == 'Squad Goalkeeping':
                        df_list_3.append(df)
                        print(stats)
                    if stats == 'Squad Shooting':
                        df_list_4.append(df)
                        print(stats)
                    if stats == 'Squad Miscellaneous Stats':
                        df_list_5.append(df)
                        print(stats)
                except ValueError as e:
                    print(f"Erro: {e}")
                    next


    df_all_stats = pd.concat(df_list_1, ignore_index=True).fillna(0)
    df_all_stats.columns = ajusta_coluna(df_all_stats)

    df_playing_time = pd.concat(df_list_2, ignore_index=True).fillna(0)
    df_playing_time.columns = ajusta_coluna(df_playing_time)

    df_goalkeeping = pd.concat(df_list_3, ignore_index=True).fillna(0)
    df_goalkeeping.columns = ajusta_coluna(df_goalkeeping)

    df_shooting = pd.concat(df_list_4, ignore_index=True).fillna(0)
    df_shooting.columns = ajusta_coluna(df_shooting)

    df_miscellaneous_stats = pd.concat(df_list_5, ignore_index=True).fillna(0)
    df_miscellaneous_stats.columns = ajusta_coluna(df_miscellaneous_stats)

    return df_all_stats,df_playing_time,df_goalkeeping,df_shooting,df_miscellaneous_stats


In [9]:
def get_team_links(list_country=['Standard Stats', 'Miscellaneous Stats', 'Goalkeeping'], stats_list=['England', 'Spain', 'France', 'Germany', 'Italy', 'Brazil']):

    df_list_1 = []
    requisicoes = 0
    df_list_2 = []

    delay = randint(5, 30)

    df_all = get_comps_link(list_country)


    links_dict = {}
    comps_dict_final = {}

    d = 0  # contador para os campeonatos

    for i, l in df_all.iterrows():
      #  sleep(delay)
        if l['Tier'] != 'Cup' and l['Gender'] == 'M':

            stats_link = l['stats_link']
            season = l['Last Season']

           # if '-' in l['Last Season']:
           #   season = '2022-2023'
           # else:
           #   season = '2022'



            if  l['Last Season'] == '2024':
              season = '2023'
            else:
              season =  l['Last Season']

            access_link = r'https://fbref.com' + stats_link.replace('()', season)

          #  print(access_link)

            info_comp_dict = {}
            info_comp_dict['Season'] = season
            info_comp_dict['Link'] = access_link

            comp = l['Comp']
            links_dict[comp] = info_comp_dict

            ua = UserAgent()
            header = {'User-Agent': str(ua.random)}


            if requisicoes <= 20:
              data = requests.get(access_link, headers=header)
              status = data.raise_for_status()  # Adiciona isso para verificar se há erros na solicitação
              requisicoes = requisicoes +1

              if status != None:
                tentativa = 1
                while status != None or tentativa <= 3:
                  print(f"Erro na solicitação para {access_link}: {status} - Tentando Novamente - Tentativa {tentativa} de 3")
                  data = requests.get(access_link, headers=header)
                  status = data.raise_for_status()
                  requisicoes = requisicoes +1
                  tentativa = tentativa + 1
                  time.sleep(1)

                if status != None:
                  print(f"Erro na solicitação do {access_link} - Indo para o próximo")
                  time.sleep(5)
                  continue
            else:
              print('Número de requests por minuto esgotado - Reiniciando')
              requisicoes  = 0
              time.sleep(60)

              data = requests.get(access_link, headers=header)
              status = data.raise_for_status()  # Adiciona isso para verificar se há erros na solicitação
              requisicoes = requisicoes +1

              if status != None:
                tentativa = 1
                while status != None or tentativa <= 3:
                  print(f"Erro na solicitação para {access_link}: {status} - Tentando Novamente - Tentativa {tentativa} de 3")
                  data = requests.get(access_link, headers=header)
                  status = data.raise_for_status()
                  requisicoes = requisicoes +1
                  tentativa = tentativa + 1
                  time.sleep(1)

                if status != None:
                  print(f"Erro na solicitação do {access_link} - Indo para o próximo")
                  time.sleep(5)
                  continue            
                       
            try:
                soup = BeautifulSoup(data.text, 'html.parser')
                champ_table = soup.select('table.stats_table')[0]
                dict_links_comp = set()
                links_teams = [f'https://fbref.com{l}' for l in [l.get('href') for l in champ_table.find_all('a')] if '/squads/' in l]

                for team in links_teams:
                  try:
                    padrao_ano = re.compile(r'\b\d{4}\b')
                    correspondencias = padrao_ano.findall(team)
                    if correspondencias:
                      ano_encontrado = correspondencias[0]
                      if '-' not in season and int(ano_encontrado) < 2022:
                        continue
                      elif '-' not in season and int(ano_encontrado) >= 2022:
                        dict_links_comp.add(team)
                    elif '-' not in season:
                      partes_caminho = team.split('/')
                      partes_caminho.insert(-1,'2023')
                      team = '/'.join(partes_caminho)
                      dict_links_comp.add(team)
                    else:
                      dict_links_comp.add(team)
                  except:
                    continue


                dict_links_comp = list(dict_links_comp)
                comps_dict_final[comp] = dict_links_comp
                d = d + 1

                print("Get Links Completo: Liga {} {} - {} de {}".format(comp,season, d, list(df_all.shape)[0] -1 ))

            except Exception as e:
                print(f"Erro durante o processamento de {comp}: {e}")
                # Aqui você pode decidir se deseja continuar para a próxima iteração ou parar completamente
                continue


    return comps_dict_final



In [10]:
def get_team_stats(list_country=['England', 'Spain', 'France', 'Germany', 'Italy', 'Brazil'], stats_list= ['Standard Stats', 'Miscellaneous Stats', 'Goalkeeping']):

    dict_links,dict_country = get_team_links(list_country, stats_list)
    delay = randint(30,60)
    time.sleep(delay)


    df_list_1 = []
    df_list_2 = []

    ii = 1
    requisicoes = 0

    ua = UserAgent()
    header = {'User-Agent': str(ua.random)}

    for key,value in dict_links.items():


      comp = key
      links = value

      print("{}/{} competiçoes".format(ii, len(list(dict_links.keys()))))

      ii = ii + 1


      for link in links:

      #  print(link)

        match = re.search(r'/([^/]+)-Stats$', link)
        ext_texto = match.group(1)
        nome_clube = ext_texto.replace('-',' ')

        k = links.index(link) + 1


        if requisicoes <= 20:
          try:
            data2 = requests.get(link, headers=header)

            if data2.status_code == 500:
              print("{} - {}/{} times - Erro 500: Internal Server Error".format(comp, k, len(links)))
              requisicoes = requisicoes + 1
              continue
            elif data2.status_code == 404:
              tentativa = 1

              while data2.status_code == 404 or tentativa <= 3:
                print(f"Erro na solicitação para {link}: {data2.status_code} - Tentando Novamente - Tentativa {tentativa} de 3")
                data2 = requests.get(link, headers=header)
                requisicoes = requisicoes + 1
                tentativa = tentativa + 1

              if data2.status_code == 404:
                print("{} - {}/{} times - Erro 404 Client Error".format(comp, k, len(links)))
                #requisicoes = requisicoes + 1
                continue

            else:
              status2 = data2.raise_for_status()
              requisicoes = requisicoes + 1

              if status2 != None:
                tentativa = 1

                while status2 != None or tentativa <= 3:
                  print(f"Erro na solicitação para {link}: {status2} - Tentando Novamente - Tentativa {tentativa} de 3")
                  data2 = requests.get(link, headers=header)
                  status2 = data2.raise_for_status()
                  requisicoes = requisicoes + 1
                  tentativa = tentativa + 1
                  time.sleep(1)

                if status2 != None:
                  print(f"Erro na solicitação do {link} - Indo para o próximo")
                  time.sleep(5)
                  continue

          except ValueError as e:
            print(f"Erro na solicitação do {link} - Indo para o próximo")
            time.sleep(5)
            continue

        else:
          hora = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
          print('Número de requests por minuto esgotado - Reiniciando - {}'.format(hora))
          requisicoes  = 0
          time.sleep(60)
                          #  next



        for stats in stats_list:
          try:
            with data2 as data:
              df = pd.read_html(data.text, match=stats)[0]


              if stats == 'Standard Stats':
                df.columns = ajusta_coluna(df)
                df_all_stats = df
                df_all_stats['Club'] = nome_clube
                df_all_stats['Competition'] = comp

              elif stats == 'Goalkeeping':
                df.columns = ajusta_coluna(df)
                df_gk = df
                df_gk['Club'] = nome_clube
                df_gk['Competition'] = comp

              elif stats == 'Advanced Goalkeeping':
                df.columns = ajusta_coluna(df)

                df_gk_columns = list(df_gk.columns)
                column_final = [item for item in list(df.columns) if item not in df_gk_columns]
                column_final.append('Player')

                df = df.loc[:, df.columns.isin(column_final)]
                df_gk = pd.merge(df_gk, df, how="left", on='Player')

              else:
                if 'df_all_stats' not in locals():
                  df.columns = ajusta_coluna(df)
                  df_all_stats = df
                  df_all_stats['Club'] = nome_clube
                  df_all_stats['Competition'] = comp
                else:
                  df.columns = ajusta_coluna(df)
              #    columns_renamed = ['_'.join([stats, item]) for item in list(df.columns)]
              #    df.columns = columns_renamed

                  df_all_stats_columns = list(df_all_stats.columns)
                  #Acresentando o nome da estatistica + abertura
                  column_final = [item for item in list(df.columns) if item not in df_all_stats_columns]

                  column_final.append('Player')
                  df = df.loc[:, df.columns.isin(column_final)]
                  df_all_stats = pd.merge(df_all_stats, df, how="left", on='Player')

              df_list_1.append(df_all_stats)

              if 'df_gk' in locals():
                df_list_2.append(df_gk)

          except ValueError as e:
            #print(f"Erro: {e} Link: {link}")
            continue

        print("{} - {}/{} times".format(comp, k, len(links)))





    print('Extração Terminada')

    try:

      df_all_stats = pd.concat(df_list_1, ignore_index=True).fillna(0)
      df_all_stats['Nation'] = df_all_stats['Nation'].astype('str')
      df_all_stats['Nation'] = [nation.split(' ')[1] if len(nation.split(' ')) > 1 else 'N/I' for nation in df_all_stats['Nation']]
    #  df_all_stats['Nation'] = df_all_stats['Nation'].map(dict_country)
      df_all_stats = df_all_stats[(df_all_stats['Player'] != 'Opponent Total') & (df_all_stats['Player'] != 'Squad Total')]

      df_all_stats.to_csv('/content/drive/MyDrive/df_all_stats_2023_2024_2.csv')

      if 'df_gk' in locals():
        df_gk = pd.concat(df_list_2, ignore_index=True).fillna(0)
        df_gk['Nation'] = df_gk['Nation'].astype('str')
        df_gk['Nation'] = [nation.split(' ')[1] if len(nation.split(' ')) > 1 else 'N/I' for nation in df_gk['Nation']]
    #    df_gk['Nation'] = df_gk['Nation'].map(dict_country)
        df_gk = df_gk[(df_all_stats['Player'] != 'Opponent Total') & (df_gk['Player'] != 'Squad Total')]
        df_gk.to_csv('/content/drive/MyDrive/df_gk_2023_2024_2.csv')

        return df_all_stats,df_gk
      else:
        return df_all_stats,[]

    except ValueError as e:
      print(f"Erro: {e} ")
      return [],[]



In [11]:
def read_table_stats(df,stats):

  list_stats = []

  # Código específico para 'Advanced Goalkeeping'
  df.columns = ajusta_coluna(df)
  df_columns = list(df.columns)
  list_stats.append(df)

  return list_stats




In [12]:
def read_stats(data,stats_list,comp,nome_clube):

  try:
    df_std_stats = pd.read_html(data.text, match='Standard Stats')[0]
    df_std_stats.columns = ajusta_coluna(df_std_stats)
    df_std_stats['Club'] = nome_clube
    df_std_stats['Competition'] = comp
    list_std_stats = []
    list_std_stats.append(df_std_stats)

    df_all_plyr = df_std_stats[['Player','Nation','Competition','Club','Age','Pos']]
    list_plyrs = []
    list_plyrs.append(df_all_plyr)


    list_df_stats = {}
    list_df_stats['All Players'] = list_plyrs
    list_df_stats['Standard Stats'] = list_std_stats

  except ValueError as e:
#    print(f"Erro: {e} --  Clube {nome_clube} --- Competição {comp}")
    return {}


  for stats in stats_list:
    if stats not in list_df_stats.keys():
      try:
        with data as data:
          df = pd.read_html(data.text, match=stats)[0]

          list_stats = read_table_stats(df,stats)
          list_df_stats[stats] = list_stats
      except ValueError as e:
      #    print(f"Erro: {e} --  Clube {nome_clube} --- Competição {comp}")
          continue
    else:
      continue

  return list_df_stats


In [13]:

def create_dataframe(dict):

    list_df_1 = []
    list_df_2 = []
    list_df_3 = []
    list_df_4 = []
    list_df_5 = []
    list_df_6 = []
    list_df_7 = []
    list_df_8 = []
    list_df_9 = []
    list_df_10 = []
    list_df_11 = []
    list_df_12 = []

    comps = list(dict.keys())

    for comp in comps:
      teams = list(dict[comp].keys())
      for team in teams:
        statistics = list(dict[comp][team].keys())

        for stats in statistics:
          df = dict[comp][team][stats][0]
        # df['Nation'] = df['Nation'].astype('str')
        # df['Nation'] = [nation.split(' ')[1] if len(nation.split(' ')) > 1 else 'N/I' for nation in df['Nation']]

          #Tratando campo de Idade
          df['Age'] = df['Age'].astype('str')
          df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
          df['Age'] = [age.split('-')[0] if '-' in age else age for age in df['Age']]
          df['Age'] = [age.split('.')[0] if '.' in age else age for age in df['Age']]
          df['Age'] = df['Age'].astype('int')

          if 'Competition' not in list(df.columns) and 'Club' not in list(df.columns):
            df['Competition'] = comp
            df['Club'] = team

          if stats == 'All Players':
            list_df_1.append(df)
          elif stats == 'Standard Stats':
            list_df_2.append(df)
          elif stats == 'Miscellaneous Stats':
            list_df_3.append(df)
          elif stats == 'Goalkeeping':
            list_df_4.append(df)
          elif stats == 'Playing Time':
            list_df_5.append(df)
          elif stats == 'Possession':
            list_df_6.append(df)
          elif stats == 'Defensive Actions':
            list_df_7.append(df)
          elif stats == 'Advanced Goalkeeping':
            list_df_8.append(df)
          elif stats == 'Shooting':
            list_df_9.append(df)
          elif stats == 'Passing':
            list_df_10.append(df)
          elif stats == 'Pass Types':
            list_df_11.append(df)
          elif stats == 'Goal and Shot Creation':
            list_df_12.append(df)


    df_all_plyr = pd.concat(list_df_1, ignore_index=True).fillna('N/I')
    df_all_plyr = df_all_plyr[(df_all_plyr['Player'] != 'Opponent Total') & (df_all_plyr['Player'] != 'Squad Total')].reset_index().drop(columns = 'index').fillna(0)


    df_gk1 = pd.concat(list_df_4, ignore_index=True).fillna(0)
    df_gk_stats = df_gk1#.drop(columns = 'Matches')
    df_gk_stats = df_gk_stats[(df_gk_stats['Player'] != 'Opponent Total') & (df_gk_stats['Player'] != 'Squad Total')].reset_index().drop(columns = 'index')

    if len(list_df_8) > 0:
      df_gk2 = pd.concat(list_df_8, ignore_index=True).fillna(0)
      df_gk_stats = pd.merge(df_gk1, df_gk2, how="left", on=['Player','Nation','Pos','Age','Competition','Club'])#.drop(columns = 'Matches')
      df_gk_stats = df_gk_stats[(df_gk_stats['Player'] != 'Opponent Total') & (df_gk_stats['Player'] != 'Squad Total')].reset_index().drop(columns = 'index')



    lista_controle = ['list_df_3','list_df_5','list_df_6','list_df_7','list_df_9','list_df_10','list_df_11','list_df_12']
   # print(type(list_df_10))
    lista_globals = []
    for nome  in lista_controle:
      lista = locals().get(nome)
      if isinstance(lista, list) and len(lista) > 0:
        lista_globals.append(nome)
        #print(lista_globals)
      else:
        continue



    colunas_para_juncao = ['Player', 'Nation', 'Pos', 'Age', 'Competition', 'Club']
    merged_df = pd.concat(list_df_2, ignore_index=True).fillna(0)
    merged = []

    for df in lista_globals:
      merged_columns = list(merged_df.columns)
      dfrme = pd.concat(locals().get(df, []), ignore_index=True).fillna(0)
      column_final = [item for item in list(dfrme.columns) if item not in merged_columns]
      column_final.extend(colunas_para_juncao)
      dfrme = dfrme.loc[:, dfrme.columns.isin(column_final)]
      merged_df = pd.merge(merged_df, dfrme, how='left', on=colunas_para_juncao)


#    merged_df = merged_df.drop(columns = 'Matches')
    merged_df = merged_df[(merged_df['Player'] != 'Opponent Total') & (merged_df['Player'] != 'Squad Total')].reset_index().drop(columns = 'index').fillna(0)


    return df_all_plyr,merged_df,df_gk_stats



In [14]:
def get_team_stats(list_country=None, stats_list = ['Standard Stats', 'Miscellaneous Stats', 'Goalkeeping']):

    dict_links = get_team_links(list_country, stats_list)
    delay = randint(30,60)
    time.sleep(delay)

    ii = 1
    requisicoes = 0

    ua = UserAgent()
    header = {'User-Agent': str(ua.random)}
    dict_comp = {}

    for key,value in dict_links.items():


      comp = key
      dict_club = {}
      links = list(set(value))



      print("{}/{} competiçoes".format(ii, len(list(dict_links.keys()))))

      ii = ii + 1


      for link in links:

        match = re.search(r'/([^/]+)-Stats$', link)
        ext_texto = match.group(1)
        nome_clube = ext_texto.replace('-',' ')
      #  dict_club = {}

        k = links.index(link) + 1
        print("{} - {}/{} times".format(comp, k, len(links)))


        if requisicoes <= 20:
          try:
            data = requests.get(link, headers=header)
            requisicoes = requisicoes + 1

            if data.status_code == 500:
              print("{} - {}/{} times - Erro 500: Internal Server Error".format(comp, k, len(links)))
              requisicoes = requisicoes + 1
              continue

            elif data.status_code in (403,404):
              tentativa = 1
              while data.status_code in (403,404) or tentativa <= 3:
                print(f"Erro na solicitação para {link}: {data.status_code} - Tentando Novamente - Tentativa {tentativa} de 3")
                data = requests.get(link, headers=header)
                requisicoes = requisicoes + 1
                tentativa = tentativa + 1

              if data.status_code in (403,404):
                print("{} - {}/{} times - Erro {}".format(comp, k, len(links),data.status_code ))
                continue
              else:
                dict_stats = read_stats(data,stats_list,comp,nome_clube)
            #    if len(dict_stats) > 0:
            #      dict_club[nome_clube] = dict_stats
            else:
              dict_stats = read_stats(data,stats_list,comp,nome_clube)
             # if len(dict_stats) > 0:
             #     dict_club[nome_clube] = dict_stats

          except ValueError as e:
            print(f"Erro na solicitação do {link} - Indo para o próximo")
            time.sleep(5)
            continue

        else:
          hora = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
          print('Número de requests por minuto esgotado - Reiniciando - {}'.format(hora))
          requisicoes  = 0
          time.sleep(60)

          try:
            data = requests.get(link, headers=header)
            requisicoes = requisicoes + 1

            if data.status_code == 500:
              print("{} - {}/{} times - Erro 500: Internal Server Error".format(comp, k, len(links)))
              requisicoes = requisicoes + 1
              continue

            elif data.status_code in (403,404):
              tentativa = 1
              while data.status_code in (403,404) or tentativa <= 3:
                print(f"Erro na solicitação para {link}: {data.status_code} - Tentando Novamente - Tentativa {tentativa} de 3")
                data = requests.get(link, headers=header)
                requisicoes = requisicoes + 1
                tentativa = tentativa + 1

              if data.status_code in (403,404):
                print("{} - {}/{} times - Erro {}".format(comp, k, len(links),data.status_code ))
                continue
              else:
                dict_stats = read_stats(data,stats_list,comp,nome_clube)

             #   if len(dict_stats) > 0:
             #     dict_club[nome_clube] = dict_stats
            else:
              dict_stats = read_stats(data,stats_list,comp,nome_clube)
            #  if len(dict_stats) > 0:
            #    dict_club[nome_clube] = dict_stats

          except ValueError as e:
            print(f"Erro na solicitação do {link} - Indo para o próximo")
            time.sleep(5)
            continue

        if len(dict_stats) > 0:
         # print(dict_stats)
          dict_club[nome_clube] = dict_stats
          #print(dict_club.keys())

      dict_comp[comp] = dict_club


    df_all_plyr,merged_df,df_gk_stats = create_dataframe(dict_comp)
    df_all_plyr.to_csv('df_all_plyr.csv')
    merged_df.to_csv('merged_df.csv')
    df_gk_stats.to_csv('df_gk_stats.csv')
    return df_all_plyr,merged_df,df_gk_stats

In [ ]:
get_team_links(list_country=['Canada'],stats_list= ['Standard Stats', 'Miscellaneous Stats', 'Goalkeeping','Playing Time','Possession','Defensive Actions','Advanced Goalkeeping','Shooting','Passing','Pass Types','Goal and Shot Creation'])

Get Links Completo: Liga Canadian Premier League 2023 - 1 de 0


{'Canadian Premier League': ['https://fbref.com/en/squads/ccdb10ba/2023/Atletico-Ottawa-Stats',
  'https://fbref.com/en/squads/cc00daff/2023/Valour-FC-Stats',
  'https://fbref.com/en/squads/1ba245db/2023/Vancouver-FC-Stats',
  'https://fbref.com/en/squads/47eaa5a1/2023/HFX-Wanderers-Stats',
  'https://fbref.com/en/squads/e084d752/2023/Pacific-FC-Stats',
  'https://fbref.com/en/squads/8514c671/2023/Forge-FC-Stats',
  'https://fbref.com/en/squads/b0e76152/2023/Cavalry-FC-Stats',
  'https://fbref.com/en/squads/6ec65eed/2023/York9-FC-Stats']}

In [15]:
df_all_plyr,merged_df,df_gk_stats = get_team_stats(list_country=['Italy'],stats_list= ['Standard Stats', 'Miscellaneous Stats', 'Goalkeeping','Playing Time','Possession','Defensive Actions','Advanced Goalkeeping','Shooting','Passing','Pass Types','Goal and Shot Creation'])

Get Links Completo: Liga Serie A 2023-2024 - 1 de 4
Get Links Completo: Liga Serie B 2023-2024 - 2 de 4
1/2 competiçoes
Serie A - 1/20 times
Serie A - 2/20 times
Serie A - 3/20 times
Serie A - 4/20 times
Serie A - 5/20 times
Serie A - 6/20 times
Serie A - 7/20 times
Serie A - 8/20 times
Serie A - 9/20 times
Serie A - 10/20 times
Serie A - 11/20 times
Serie A - 12/20 times
Serie A - 13/20 times
Serie A - 14/20 times
Serie A - 15/20 times
Serie A - 16/20 times
Serie A - 17/20 times
Serie A - 18/20 times
Serie A - 19/20 times
Serie A - 20/20 times
2/2 competiçoes
Serie B - 1/19 times
Serie B - 2/19 times
Número de requests por minuto esgotado - Reiniciando - 2024-01-23 11:43:58
Serie B - 3/19 times
Serie B - 4/19 times
Serie B - 5/19 times
Serie B - 6/19 times
Serie B - 7/19 times
Serie B - 8/19 times
Serie B - 9/19 times
Serie B - 10/19 times
Serie B - 11/19 times
Serie B - 12/19 times
Serie B - 13/19 times
Serie B - 14/19 times
Serie B - 15/19 times
Serie B - 16/19 times
Serie B - 17/19

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = [age.split('-')[0] if '-' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = [age.split('-')[0] if '-' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = [age.split('.')[0] if '.' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:33: SettingWithCopyWarning: 
A value is trying to be set on a cop

...

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = [age.split('.')[0] if '.' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('int')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = [age.split('.')[0] if '.' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('int')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = [age.split('.')[0] if '.' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('int')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_16220\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [24]:
df_all_plyr,merged_df,df_gk_stats = get_team_stats(list_country=['Argentina'],stats_list= ['Standard Stats', 'Miscellaneous Stats', 'Goalkeeping','Playing Time','Possession','Defensive Actions','Advanced Goalkeeping','Shooting','Passing','Pass Types','Goal and Shot Creation'])

Get Links Completo: Liga Argentine Primera División 2023 - 1 de 1
1/1 competiçoes
Argentine Primera División - 1/28 times
Argentine Primera División - 2/28 times
Argentine Primera División - 3/28 times
Argentine Primera División - 4/28 times
Argentine Primera División - 5/28 times
Argentine Primera División - 6/28 times
Argentine Primera División - 7/28 times
Argentine Primera División - 8/28 times
Argentine Primera División - 9/28 times
Argentine Primera División - 10/28 times
Argentine Primera División - 11/28 times
Argentine Primera División - 12/28 times
Argentine Primera División - 13/28 times
Argentine Primera División - 14/28 times
Argentine Primera División - 15/28 times
Argentine Primera División - 16/28 times
Argentine Primera División - 17/28 times
Argentine Primera División - 18/28 times
Argentine Primera División - 19/28 times
Argentine Primera División - 20/28 times
Argentine Primera División - 21/28 times
Argentine Primera División - 22/28 times
Número de requests por mi

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = [age.split('-')[0] if '-' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = [age.split('.')[0] if '.' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:33: SettingWithCopyWarning: 
A value is trying to be set on a cop

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype('str')
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ['0' if 'nan' in age else age for age in df['Age']]
C:\Users\Soeiro\AppData\Local\Temp\ipykernel_10416\1397507138.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [ ]:
leagues

,Tier,Comp,Country,Gender,Seasons,First Season,Last Season
0,Cup,FA Cup,England,M,10,2014-2015,2023-2024
1,Cup,FA Community Shield,England,M,9,2015-2016,2023-2024
2,Cup,EFL Cup,England,M,10,2014-2015,2023-2024
3,Cup,Copa del Rey,Spain,M,10,2014-2015,2023-2024
4,Cup,Supercopa de España,Spain,M,9,2015-2016,2023-2024
...,...,...,...,...,...,...,...
140,Third Division,UEFA Europa Conference League,N/I,M,3,2021-2022,2023-2024
141,Third Division,USL Championship,United States,M,14,2011,2024
142,Third Division,USL League One,United States,M,5,2020,2024
143,Fourth Division,EFL League Two,England,M,22,2002-2003,2023-2024


In [ ]:
teste = pd.read_csv('/content/drive/MyDrive/df_all_stats_2023_2024_2.csv')
teste.head(5)

,Unnamed: 0,Player,Nation,Pos,Age,MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,...,Performance Int,Performance TklW,Performance PKwon,Performance PKcon,Performance OG,Performance Recov,Aerial Duels Won,Aerial Duels Lost,Aerial Duels Won%,Playing Time MP
0,0,Joachim Andersen,dk DEN,DF,27-224,20.0,20.0,1800.0,20.0,2.0,...,21.0,19.0,0.0,0.0,0.0,96.0,45.0,24.0,65.2,0.0
1,1,Marc Guéhi,eng ENG,DF,23-181,19.0,19.0,1709.0,19.0,0.0,...,15.0,15.0,0.0,0.0,0.0,103.0,24.0,22.0,52.2,0.0
2,2,Tyrick Mitchell,eng ENG,DF,24-131,19.0,19.0,1627.0,18.1,1.0,...,15.0,24.0,0.0,0.0,0.0,110.0,8.0,11.0,42.1,0.0
3,3,Jordan Ayew,gh GHA,"FW,MF",32-121,19.0,18.0,1546.0,17.2,2.0,...,11.0,21.0,0.0,0.0,0.0,100.0,23.0,35.0,39.7,0.0
4,4,Sam Johnstone,eng ENG,GK,30-291,16.0,16.0,1436.0,16.0,0.0,...,0.0,0.0,0.0,1.0,0.0,10.0,1.0,0.0,100.0,0.0


In [ ]:
teste['Nation'] = [nation.split(' ')[1] if len(nation.split(' ')) > 1 else 'N/I' for nation in teste['Nation']]


In [ ]:
teste.head()

,Unnamed: 0,Player,Nation,Pos,Age,MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,...,Performance Int,Performance TklW,Performance PKwon,Performance PKcon,Performance OG,Performance Recov,Aerial Duels Won,Aerial Duels Lost,Aerial Duels Won%,Playing Time MP
0,0,Joachim Andersen,DEN,DF,27-224,20.0,20.0,1800.0,20.0,2.0,...,21.0,19.0,0.0,0.0,0.0,96.0,45.0,24.0,65.2,0.0
1,1,Marc Guéhi,ENG,DF,23-181,19.0,19.0,1709.0,19.0,0.0,...,15.0,15.0,0.0,0.0,0.0,103.0,24.0,22.0,52.2,0.0
2,2,Tyrick Mitchell,ENG,DF,24-131,19.0,19.0,1627.0,18.1,1.0,...,15.0,24.0,0.0,0.0,0.0,110.0,8.0,11.0,42.1,0.0
3,3,Jordan Ayew,GHA,"FW,MF",32-121,19.0,18.0,1546.0,17.2,2.0,...,11.0,21.0,0.0,0.0,0.0,100.0,23.0,35.0,39.7,0.0
4,4,Sam Johnstone,ENG,GK,30-291,16.0,16.0,1436.0,16.0,0.0,...,0.0,0.0,0.0,1.0,0.0,10.0,1.0,0.0,100.0,0.0


In [ ]:
df_all_stats_2 = df_all_stats.copy()

In [ ]:
df_new = pd.merge(df_all_stats_2,leagues[['Comp','Country']],how = 'left', left_on = 'Competition',right_on = 'Comp')

In [ ]:
df_new.drop_duplicates(keep=False, inplace=True)

In [ ]:
df_new.shape

(33379, 54)

In [ ]:
df_new = df_new.head().drop(columns = 'Comp')

,Player,Nation,Pos,Age,MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,Performance Ast,...,Performance TklW,Performance PKwon,Performance PKcon,Performance OG,Performance Recov,Aerial Duels Won,Aerial Duels Lost,Aerial Duels Won%,Playing Time MP,Country
0,Joachim Andersen,dk DEN,DF,27-224,20.0,20.0,1800.0,20.0,2.0,1.0,...,19.0,0.0,0.0,0.0,96.0,45.0,24.0,65.2,0.0,England
1,Marc Guéhi,eng ENG,DF,23-181,19.0,19.0,1709.0,19.0,0.0,1.0,...,15.0,0.0,0.0,0.0,103.0,24.0,22.0,52.2,0.0,England
2,Tyrick Mitchell,eng ENG,DF,24-131,19.0,19.0,1627.0,18.1,1.0,2.0,...,24.0,0.0,0.0,0.0,110.0,8.0,11.0,42.1,0.0,England
3,Jordan Ayew,gh GHA,"FW,MF",32-121,19.0,18.0,1546.0,17.2,2.0,5.0,...,21.0,0.0,0.0,0.0,100.0,23.0,35.0,39.7,0.0,England
4,Sam Johnstone,eng ENG,GK,30-291,16.0,16.0,1436.0,16.0,0.0,0.0,...,0.0,0.0,1.0,0.0,10.0,1.0,0.0,100.0,0.0,England


In [ ]:
df_new[df_new['Player'] == 'Adam Laszczynski']['Club']

5266    Cottbus U17
5290          Monza
5291          Monza
Name: Club, dtype: object

In [ ]:
!pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 17.7 MB/s eta 0:00:00


In [ ]:
import pycountry

def mark_continent(df):


    # Mapeia cada nome de país para o código de país ISO-3166-1 alpha-2 correspondente
    country_codes = {}
    for country in pycountry.countries:
        country_codes[country.name] = country.alpha_2

    # Adiciona uma nova coluna "code" para o DataFrame com códigos de países correspondentes
    df['code'] = df['Country'].apply(lambda x: country_codes.get(x))

    # Usa a biblioteca pycountry para obter informações de continentes correspondentes aos códigos de países
    continents = {}
    for country in pycountry.countries:
        continents[country.alpha_2] = country.continent.name

    # Adiciona uma nova coluna "continent" ao DataFrame com continentes correspondentes
    df['continent'] = df['code'].apply(lambda x: continents.get(x))

    # Remove a coluna "code" do DataFrame
    df.drop('code', axis=1, inplace=True)

    return df

In [ ]:
df_new[df_new['code'] == 'None']['Country'].unique()

array(['England', 'Bolivia', 'China PR', 'Czech Republic', 'Iran',
       'Korea Republic', 'Russia', 'Scotland', 'Turkey', 'Venezuela'],
      dtype=object)

In [ ]:
no_code_country = ['England', 'Bolivia', 'China PR', 'Czech Republic', 'Iran','Korea Republic', 'Russia', 'Scotland', 'Turkey', 'Venezuela']
code_no_code = ['EN','BO','CN','CZ',  'IR','KR','RU', 'SC','TR','VE']

dict_no_code = dict(zip(no_code_country, code_no_code))

# Mapeia cada nome de país para o código de país ISO-3166-1 alpha-2 correspondente
country_codes = {}
for country in pycountry.countries:
  country_codes[country.name] = country.alpha_2

country_codes.update(dict_no_code)

In [ ]:
# Adiciona uma nova coluna "code" para o DataFrame com códigos de países correspondentes
df_new['code'] = df_new['Country'].apply(lambda x: country_codes.get(x))
df_new['Country'] = df_new['Country'].astype('str')
df_new['code'] = df_new['code'].astype('str')

In [ ]:
df_new[df_new['code'] == 'None']['Country'].unique()

array([], dtype=object)

In [ ]:
df_new

,Player,Nation,Pos,Age,MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,Performance Ast,...,Performance PKcon,Performance OG,Performance Recov,Aerial Duels Won,Aerial Duels Lost,Aerial Duels Won%,Playing Time MP,Comp,Country,code
0,Joachim Andersen,dk DEN,DF,27-224,20.0,20.0,1800.0,20.0,2.0,1.0,...,0.0,0.0,96.0,45.0,24.0,65.2,0.0,Premier League,England,EN
1,Marc Guéhi,eng ENG,DF,23-181,19.0,19.0,1709.0,19.0,0.0,1.0,...,0.0,0.0,103.0,24.0,22.0,52.2,0.0,Premier League,England,EN
2,Tyrick Mitchell,eng ENG,DF,24-131,19.0,19.0,1627.0,18.1,1.0,2.0,...,0.0,0.0,110.0,8.0,11.0,42.1,0.0,Premier League,England,EN
3,Jordan Ayew,gh GHA,"FW,MF",32-121,19.0,18.0,1546.0,17.2,2.0,5.0,...,0.0,0.0,100.0,23.0,35.0,39.7,0.0,Premier League,England,EN
4,Sam Johnstone,eng ENG,GK,30-291,16.0,16.0,1436.0,16.0,0.0,0.0,...,1.0,0.0,10.0,1.0,0.0,100.0,0.0,Premier League,England,EN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39607,David Longe-King,eng ENG,DF,28-349,0.0,2.0,165.0,1.8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,National League,England,EN
39608,Flynn Clarke,sct SCO,MF,23-149,0.0,0.0,38.0,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,National League,England,EN
39609,Aaron Blair,eng ENG,FW,23-265,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,National League,England,EN
39610,Joshua Strizovic,eng ENG,GK,23-273,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,National League,England,EN


In [ ]:
!pip install pycountry_convert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.7 MB/s eta 0:00:00


In [ ]:
import pycountry_convert as pc
paises = list(df_new['code'].unique())
#code_no_code = ['EN','BO','CN','CZ',  'IR','KR','RU', 'SC','TR','VE']

continent = {}
for code in paises:
  if code != 'EN' and code != 'SC' :
    continent_name = pc.country_alpha2_to_continent_code(code)
    continent[code] = continent_name
    #print(continent_name)

In [ ]:
continent

{'ES': 'EU',
 'FR': 'EU',
 'DE': 'EU',
 'AR': 'SA',
 'AU': 'OC',
 'AT': 'EU',
 'BE': 'EU',
 'BO': 'SA',
 'BR': 'SA',
 'BG': 'EU',
 'CA': 'NA',
 'CL': 'SA',
 'CN': 'AS',
 'HR': 'EU',
 'CZ': 'EU',
 'DK': 'EU',
 'EC': 'SA',
 'FI': 'EU',
 'GR': 'EU',
 'HU': 'EU',
 'IN': 'AS',
 'IR': 'AS',
 'JP': 'AS',
 'KR': 'AS',
 'SA': 'AS',
 'MX': 'NA',
 'NL': 'EU',
 'PL': 'EU',
 'PT': 'EU',
 'RO': 'EU',
 'ZA': 'AF',
 'RU': 'EU',
 'RS': 'EU',
 'CH': 'EU',
 'TR': 'AS',
 'UA': 'EU',
 'UY': 'SA',
 'VE': 'SA',
 'IT': 'EU',
 'US': 'NA'}

In [ ]:
continent['EN'] = 'EU'
continent['SC'] = 'EU'

In [ ]:
df_new['Continent'] = df_new['code'].apply(lambda x: continent.get(x))

In [ ]:
df_new

,Player,Nation,Pos,Age,MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,Performance Ast,...,Performance OG,Performance Recov,Aerial Duels Won,Aerial Duels Lost,Aerial Duels Won%,Playing Time MP,Comp,Country,code,Continent
0,Joachim Andersen,dk DEN,DF,27-224,20.0,20.0,1800.0,20.0,2.0,1.0,...,0.0,96.0,45.0,24.0,65.2,0.0,Premier League,England,EN,EU
1,Marc Guéhi,eng ENG,DF,23-181,19.0,19.0,1709.0,19.0,0.0,1.0,...,0.0,103.0,24.0,22.0,52.2,0.0,Premier League,England,EN,EU
2,Tyrick Mitchell,eng ENG,DF,24-131,19.0,19.0,1627.0,18.1,1.0,2.0,...,0.0,110.0,8.0,11.0,42.1,0.0,Premier League,England,EN,EU
3,Jordan Ayew,gh GHA,"FW,MF",32-121,19.0,18.0,1546.0,17.2,2.0,5.0,...,0.0,100.0,23.0,35.0,39.7,0.0,Premier League,England,EN,EU
4,Sam Johnstone,eng ENG,GK,30-291,16.0,16.0,1436.0,16.0,0.0,0.0,...,0.0,10.0,1.0,0.0,100.0,0.0,Premier League,England,EN,EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39607,David Longe-King,eng ENG,DF,28-349,0.0,2.0,165.0,1.8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,National League,England,EN,EU
39608,Flynn Clarke,sct SCO,MF,23-149,0.0,0.0,38.0,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,National League,England,EN,EU
39609,Aaron Blair,eng ENG,FW,23-265,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,National League,England,EN,EU
39610,Joshua Strizovic,eng ENG,GK,23-273,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,National League,England,EN,EU


In [ ]:
conf = {
    'NA': 'CONCACAF',
    'SA': 'CONMEBOL',
    'AS': 'AFC',
    'OC': 'OFC',
    'AF': 'CAF',
    'EU': 'UEFA'
}

In [ ]:
df_new['Confederation'] = df_new['Continent'].apply(lambda x: conf.get(x))

In [ ]:
df_new

,Player,Nation,Pos,Age,MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,Performance Ast,...,Performance Recov,Aerial Duels Won,Aerial Duels Lost,Aerial Duels Won%,Playing Time MP,Comp,Country,code,Continent,Confederation
0,Joachim Andersen,dk DEN,DF,27-224,20.0,20.0,1800.0,20.0,2.0,1.0,...,96.0,45.0,24.0,65.2,0.0,Premier League,England,EN,EU,UEFA
1,Marc Guéhi,eng ENG,DF,23-181,19.0,19.0,1709.0,19.0,0.0,1.0,...,103.0,24.0,22.0,52.2,0.0,Premier League,England,EN,EU,UEFA
2,Tyrick Mitchell,eng ENG,DF,24-131,19.0,19.0,1627.0,18.1,1.0,2.0,...,110.0,8.0,11.0,42.1,0.0,Premier League,England,EN,EU,UEFA
3,Jordan Ayew,gh GHA,"FW,MF",32-121,19.0,18.0,1546.0,17.2,2.0,5.0,...,100.0,23.0,35.0,39.7,0.0,Premier League,England,EN,EU,UEFA
4,Sam Johnstone,eng ENG,GK,30-291,16.0,16.0,1436.0,16.0,0.0,0.0,...,10.0,1.0,0.0,100.0,0.0,Premier League,England,EN,EU,UEFA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39607,David Longe-King,eng ENG,DF,28-349,0.0,2.0,165.0,1.8,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,National League,England,EN,EU,UEFA
39608,Flynn Clarke,sct SCO,MF,23-149,0.0,0.0,38.0,0.4,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,National League,England,EN,EU,UEFA
39609,Aaron Blair,eng ENG,FW,23-265,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,National League,England,EN,EU,UEFA
39610,Joshua Strizovic,eng ENG,GK,23-273,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,National League,England,EN,EU,UEFA


In [ ]:
df_new.to_csv('/content/drive/MyDrive/df_all_stats_2023_2024_3.csv')